In [175]:
! pip install xmltodict

In [176]:
import pandas as pd
import re
import sys
import requests
import json
import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen
import xmltodict
from tqdm import tqdm
import random
import pprint

In [177]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)
totalCnt = int(dict_contents['LawSearch']['totalCnt'])

In [179]:
rows = []
eng_cnt = 0
kor_cnt = 0
# tmp = 30 # 페이지 테스트용
total_page = int(totalCnt/20)+2
for p in tqdm(range(1,total_page)):
  url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=' + str(p)
  # url 불러오기
  response = requests.get(url)
  # XML -> 딕셔너리 변환후 파싱
  contents = response.text
  dict_contents = xmltodict.parse(contents)
  try:
    for content in dict_contents['LawSearch']['law']:
      법령명한글 = content['법령명한글']
      법령명영문 = content['법령명영문']
      mst = content['법령일련번호']

      eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      eng_response = requests.get(eng_url)
      eng_contents = eng_response.text
      eng_dict_contents = xmltodict.parse(eng_contents)

      kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      kor_response = requests.get(kor_url)
      kor_contents = kor_response.text
      kor_dict_contents = xmltodict.parse(kor_contents)
      eng_cnt += len(eng_dict_contents['Law']['JoSection']['Jo'])
      kor_cnt += len(kor_dict_contents['법령']['조문']['조문단위'])
      # 영어조문 개수와 한글 조문 개수 맞지 않다면 건너뛰기
      if len(eng_dict_contents['Law']['JoSection']['Jo']) != len(kor_dict_contents['법령']['조문']['조문단위']):
        continue
      try:
        for eng_jo, kor_jo in zip(eng_dict_contents['Law']['JoSection']['Jo'],kor_dict_contents['법령']['조문']['조문단위']):
            if eng_jo['joYn']!= 'Y':
              continue
            if eng_jo['joBrNo']!= '00':
              continue
            원문 = eng_jo['joCts']
            
            if kor_jo['조문여부']!= '조문':
              continue
            if kor_jo.get('조문가지번호'):
              continue
            jo_txt = ''
            hang_txt = ''
            ho_txt = ''
            jo_txt += kor_jo['조문내용']
            if kor_jo.get('항'):
                if '호' in kor_jo['항']:
                    for ho in kor_jo['항']['호']:
                        ho_txt+=ho['호내용']
                    hang_txt += ho_txt
                else:
                    for hang in kor_jo['항']:
                          hang_txt += hang['항내용']
                          if type(hang) == dict and '호' in hang:
                              for ho in hang['호']:
                                  ho_txt+=ho['호내용']
                              hang_txt += ho_txt
                jo_txt += hang_txt
            if '조문참고자료' in kor_jo:
                jo_txt += kor_jo['조문참고자료']
            정답번역문 = jo_txt
            rows.append({'법령명한글':법령명한글, '법령명영문':법령명영문, 'MST':mst, '원문':원문, '정답번역문':정답번역문})
      except Exception as e:
        print('eng_kor_contents 예외 발생:', e)
  except Exception as e:
    print('dict_contents 예외 발생:', e)
df = pd.DataFrame(rows)

  3%|▎         | 4/130 [02:15<1:05:19, 31.10s/it]

dict_contents 예외 발생: 'NoneType' object is not subscriptable


  5%|▍         | 6/130 [03:36<1:13:12, 35.43s/it]

eng_kor_contents 예외 발생: string indices must be integers


  9%|▉         | 12/130 [07:04<1:01:00, 31.02s/it]

...

 94%|█████████▍| 122/130 [1:16:22<03:48, 28.60s/it]

dict_contents 예외 발생: 'NoneType' object is not subscriptable


 98%|█████████▊| 127/130 [1:20:56<02:11, 43.97s/it]

dict_contents 예외 발생: 'NoneType' object is not subscriptable


100%|██████████| 130/130 [1:25:56<00:00, 39.67s/it]


In [180]:
df

,법령명한글,법령명영문,MST,원문,정답번역문
0,119구조ㆍ구급에 관한 법률,ACT ON 119 RESCUE AND EMERGENCY MEDICAL SERVICES,236231,Article 1 (Purpose) The purpose of this Act is...,"제1조(목적) 이 법은 화재, 재난ㆍ재해 및 테러, 그 밖의 위급한 상황에서 119..."
1,119구조ㆍ구급에 관한 법률,ACT ON 119 RESCUE AND EMERGENCY MEDICAL SERVICES,236231,Article 2 (Definitions) The terms used in this...,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2016.1.2...
2,119구조ㆍ구급에 관한 법률,ACT ON 119 RESCUE AND EMERGENCY MEDICAL SERVICES,236231,Article 3 (Responsibilities of the State) (1) ...,"제3조(국가 등의 책무)① 국가와 지방자치단체는 119구조ㆍ구급(이하 ""구조ㆍ구급""..."
3,119구조ㆍ구급에 관한 법률,ACT ON 119 RESCUE AND EMERGENCY MEDICAL SERVICES,236231,Article 4 (Rights and Obligations of People) (...,제4조(국민의 권리와 의무)① 누구든지 위급상황에 처한 경우에는 국가와 지방자치단체...
4,119구조ㆍ구급에 관한 법률,ACT ON 119 RESCUE AND EMERGENCY MEDICAL SERVICES,236231,Article 5 (Relationship to Other Statutes) Exc...,제5조(다른 법률과의 관계) 구조ㆍ구급활동에 관하여 다른 법률에 특별한 규정이 있는...
...,...,...,...,...,...
66639,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,"Article 10 (Request for Financial Information,...",제10조(금융정보등의 요청 및 제공)① 관할 보건소장은 법 제13조제1항에 따라 의...
66640,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,Article 11 (Requirements for Designation as Sp...,"제11조(희귀질환전문기관의 지정 요건)① 법 제14조제1항 각 호 외의 부분에서 ""..."
66641,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,Article 12 (Entrustment of Evaluation of Speci...,제12조(희귀질환전문기관 평가의 위탁)① 보건복지부장관은 법 제15조제3항에 따라 ...
66642,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,Article 13 (Delegation of Authority)\nPursuant...,제13조(권한의 위임) 보건복지부장관은 법 제22조제1항에 따라 다음 각 호의 권한...


In [184]:
df[-10:]

,법령명한글,법령명영문,MST,원문,정답번역문
66634,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,Article 5 (Detailed Operational Rules)\nIn add...,제5조(운영세칙) 이 영에서 규정한 사항 외에 위원회와 분야별 전문위원회의 구성 및...
66635,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,Article 6 (Projects for Statistics on Registra...,제6조(희귀질환 등록통계사업)① 보건복지부장관은 법 제10조제1항 전단에 따른 등록...
66636,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,"Article 7 (Details, Methods, etc. of Fact-Find...","제7조(실태조사의 내용 및 방법 등)① 법 제11조제1항에 따른 실태조사(이하 ""실..."
66637,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,Article 8 (Medical Expense Support Projects) (...,"제8조(의료비 지원 사업)① 법 제12조에 따른 의료비 지원 사업(이하 ""의료비 지..."
66638,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,"Article 9 (Scope of Financial Information, etc...","제9조(금융정보등의 범위)① 법 제13조제1항제1호에서 ""대통령령으로 정하는 자료 ..."
66639,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,"Article 10 (Request for Financial Information,...",제10조(금융정보등의 요청 및 제공)① 관할 보건소장은 법 제13조제1항에 따라 의...
66640,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,Article 11 (Requirements for Designation as Sp...,"제11조(희귀질환전문기관의 지정 요건)① 법 제14조제1항 각 호 외의 부분에서 ""..."
66641,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,Article 12 (Entrustment of Evaluation of Speci...,제12조(희귀질환전문기관 평가의 위탁)① 보건복지부장관은 법 제15조제3항에 따라 ...
66642,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,Article 13 (Delegation of Authority)\nPursuant...,제13조(권한의 위임) 보건복지부장관은 법 제22조제1항에 따라 다음 각 호의 권한...
66643,희귀질환관리법 시행령,ENFORCEMENT DECREE OF THE RARE DISEASE MANAGEM...,190356,Article 14 (Handling of Sensitive Information ...,제14조(민감정보 및 고유식별정보의 처리)① 보건복지부장관(법 제15조제3항에 따라...


In [182]:
eng_cnt

108189

In [183]:
kor_cnt

108584

In [181]:
df.to_csv('검수2.csv', index=False, encoding="utf-8-sig") 

In [ ]:
df.iloc[-10:,-2:]

,원문,정답번역문
66634,Article 5 (Detailed Operational Rules)\nIn add...,제5조(운영세칙) 이 영에서 규정한 사항 외에 위원회와 분야별 전문위원회의 구성 및...
66635,Article 6 (Projects for Statistics on Registra...,제6조(희귀질환 등록통계사업)① 보건복지부장관은 법 제10조제1항 전단에 따른 등록...
66636,"Article 7 (Details, Methods, etc. of Fact-Find...","제7조(실태조사의 내용 및 방법 등)① 법 제11조제1항에 따른 실태조사(이하 ""실..."
66637,Article 8 (Medical Expense Support Projects) (...,"제8조(의료비 지원 사업)① 법 제12조에 따른 의료비 지원 사업(이하 ""의료비 지..."
66638,"Article 9 (Scope of Financial Information, etc...","제9조(금융정보등의 범위)① 법 제13조제1항제1호에서 ""대통령령으로 정하는 자료 ..."
66639,"Article 10 (Request for Financial Information,...",제10조(금융정보등의 요청 및 제공)① 관할 보건소장은 법 제13조제1항에 따라 의...
66640,Article 11 (Requirements for Designation as Sp...,"제11조(희귀질환전문기관의 지정 요건)① 법 제14조제1항 각 호 외의 부분에서 ""..."
66641,Article 12 (Entrustment of Evaluation of Speci...,제12조(희귀질환전문기관 평가의 위탁)① 보건복지부장관은 법 제15조제3항에 따라 ...
66642,Article 13 (Delegation of Authority)\nPursuant...,제13조(권한의 위임) 보건복지부장관은 법 제22조제1항에 따라 다음 각 호의 권한...
66643,Article 14 (Handling of Sensitive Information ...,제14조(민감정보 및 고유식별정보의 처리)① 보건복지부장관(법 제15조제3항에 따라...


In [ ]:
random.randint(0,66643)

64522

In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2])
pprint.pprint(df.iloc[num,-1])

11835
('Article 12 (Power and Duties of Jurors) (1) Jurors shall have the power to '
 'find facts and present opinions on the application of Acts and subordinate '
 'statutes and the determination of punishment with respect to the case for '
 'which they take part in a participatory trial.\n'
 '  (2) Every juror shall abide by Acts and subordinate statutes and perform '
 'his/her duties independently and sincerely.\n'
 '  (3) No juror shall divulge confidential information known to him/her in '
 'the scope of his/her duties nor shall he/she commit any act of undermining '
 'fairness of a trial.')
('제12조(배심원의 권한과 의무)① 배심원은 국민참여재판을 하는 사건에 관하여 사실의 인정, 법령의 적용 및 형의 양정에 관한 의견을 '
 '제시할 권한이 있다.② 배심원은 법령을 준수하고 독립하여 성실히 직무를 수행하여야 한다.③ 배심원은 직무상 알게 된 비밀을 누설하거나 '
 '재판의 공정을 해하는 행위를 하여서는 아니 된다.')


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2])
pprint.pprint(df.iloc[num,-1])

22857
('Article 43 (Matters to be Observed by Business Operator)\n'
 '  The matters to be observed by a business operator (including a '
 'representative in the case of a corporation) and his/her employees shall be '
 'as set forth in Table 10 attached hereto. <Amended on March 22, 2018>')
('제43조(영업자의 준수사항) 영업자(법인인 경우에는 그 대표자를 포함한다)와 그 종사자의 준수사항은 별표 10과 같다. <개정 '
 '2018.3.22>')


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2])
pprint.pprint(df.iloc[num,-1])

16196
('Article 12 (Establishment of Systems for Management and Joint Use of '
 'Information on Meteorological Services) (1) The Administrator of the Korea '
 'Meteorological Administration shall establish and operate a weather '
 'information system (including incidental facilities) to facilitate the '
 'dissemination and use of information on meteorological services.\n'
 '  (2) The Administrator of the Korea Meteorological Administration may '
 'request cooperation under paragraph (1) from national agencies, local '
 'governments, and persons specified by Ordinance of the Ministry of '
 'Environment, which produce and manage information on meteorological '
 'services.\n'
 '  [This Article Wholly Amended on Dec. 31, 2008]')
('제12조(기상업무에 관한 정보의 관리 및 공동활용체계의 구축 등)① 기상청장은 기상정보시스템(그 부대시설을 포함한다)을 구축ㆍ운영하여 '
 '기상업무에 관한 정보의 보급 및 이용을 촉진시켜야 한다.② 기상청장은 기상업무에 관한 정보를 생산ㆍ관리하는 국가기관, 지방자치단체 및 '
 '환경부령으로 정하는 자에 대하여 제1항에 따른 협력을 요청할 수 있다.[전문개정 2008.12.31]')


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2])
pprint.pprint(df.iloc[num,-1])

64753
('Article 298 (Indecent Act by Compulsion) A person who, through violence or '
 'intimidation, commits an indecent act on another shall be punished by '
 'imprisonment with labor for not more than ten years or by a fine not '
 'exceeding 15 million won. <Amended on Dec. 29, 1995>')
('제298조(강제추행) 폭행 또는 협박으로 사람에 대하여 추행을 한 자는 10년 이하의 징역 또는 1천500만원 이하의 벌금에 처한다. '
 '<개정 1995.12.29>')


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2])
pprint.pprint(df.iloc[num,-1])

38753
('Article 31 (Organization of Committee) (1) The Committee shall be comprised '
 'of not more than 30 members, including two Chairpersons.\n'
 '  (2) The following persons shall be Committee members: <Amended on Jul. 26, '
 '2017; Oct. 24, 2017>\n'
 '  1. One person nominated by the Minister of Economy and Finance from among '
 'the Minister of Land, Infrastructure and Transport and the Vice Ministers of '
 'Economy and Finance; one person nominated by the Minister of Science and ICT '
 'from among the Vice Ministers of Science and ICT; one person nominated by '
 'the Minister of Culture, Sports and Tourism from among the Vice Minister of '
 'Education, the Vice Minister of Defense, the Vice Minister of the Interior '
 'and Safety, and the Vice Ministers of Culture, Sports and Tourism; one '
 'person nominated by the Minister of Trade, Industry and Energy from among '
 'the Vice Minister of Agriculture, Food and Rural Affairs and the Vice '
 'Ministers of Trade, Industry and Ener

In [ ]:
pprint.pprint(df.iloc[num,-2], width = 200)

('Article 31 (Organization of Committee) (1) The Committee shall be comprised of not more than 30 members, including two Chairpersons.\n'
 '  (2) The following persons shall be Committee members: <Amended on Jul. 26, 2017; Oct. 24, 2017>\n'
 '  1. One person nominated by the Minister of Economy and Finance from among the Minister of Land, Infrastructure and Transport and the Vice Ministers of Economy and Finance; one person nominated '
 'by the Minister of Science and ICT from among the Vice Ministers of Science and ICT; one person nominated by the Minister of Culture, Sports and Tourism from among the Vice Minister of Education, '
 'the Vice Minister of Defense, the Vice Minister of the Interior and Safety, and the Vice Ministers of Culture, Sports and Tourism; one person nominated by the Minister of Trade, Industry and '
 'Energy from among the Vice Minister of Agriculture, Food and Rural Affairs and the Vice Ministers of Trade, Industry and Energy; and the Vice Minister of Environme

In [ ]:
pprint.pprint(df.iloc[num,-1], width = 100)

('제31조(위원회의 조직)① 위원회는 위원장 2명을 포함한 30명 이내의 위원으로 구성한다.② 위원회의 위원은 다음 각 호의 사람이 된다. <개정 2017.7.26, '
 '2017.10.24>1. 국토교통부장관, 기획재정부차관 중 기획재정부장관이 지명하는 사람 1명, 과학기술정보통신부차관 중 과학기술정보통신부장관이 지명하는 사람 1명, '
 '교육부차관, 국방부차관, 행정안전부차관, 문화체육관광부차관 중 문화체육관광부장관이 지명하는 사람 1명, 농림축산식품부차관, 산업통상자원부차관 중 산업통상자원부장관이 지명하는 '
 '사람 1명, 환경부차관 및 건설청장2. 행정중심복합도시 건설에 필요한 학식과 경험이 풍부한 사람 중에서 건설청장의 추천을 받아 국토교통부장관이 위촉하는 사람③ 위원장은 '
 '국토교통부장관과 제2항제2호에 따른 위원 중에서 대통령이 위촉하는 사람이 된다.④ 위원장은 위원회에 상정된 안건과 관련된 관계 기관 및 지방자치단체의 장 등을 회의에 '
 '참석하여 발언하게 할 수 있다.⑤ 제2항제2호에 따른 위원의 임기는 2년으로 하며, 세 차례만 연임할 수 있다. <개정 2018.12.31>⑥ 위원회에 간사위원 1명을 '
 '두며, 간사위원은 건설청장이 된다.[전문개정 2014.6.11]')


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

12833
('Article 2 (Definition) The term "international cultural exchanges" in this Act means international cooperation and activities provided or performed to enhance mutual understanding of culture '
 '(referring to culture in a broad sense encompassing art, tourism, sports, etc.) among countries in relevant fields, such as culture, art, tourism, and sports under any of the following statutes: '
 '<Amended on Oct. 16, 2018>\n'
 '  1. The Framework Act on Culture;\n'
 '  2. The Framework Act on the Promotion of Cultural Industries;\n'
 '  3. The Framework Act on the Video Industry Promotion;\n'
 '  4. The Framework Act on Korean Language;\n'
 '  5. The Culture and Arts Promotion Act;\n'
 '  6. The Content Industry Promotion Act;\n'
 '  7. The Promotion of the Motion Pictures and Video Products Act;\n'
 '  8. The Publishing Industry Promotion Act;\n'
 '  9. The Print Culture Industry Promotion Act;\n'
 '  10. The Craft Industry Promotion Act;\n'
 '  11. The Museum and Art Gallery Suppor

In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

55731
('Article 5 (Scope of Heads of Relevant Central Administrative Agencies) "The heads of the relevant central administrative agencies prescribed by Presidential Decree" in Article 5 (3) of the Act '
 'means the heads of the following institutions:\n'
 '  1. The Ministry of Economy and Finance;\n'
 '  2. The Ministry of Education;\n'
 '  3. The Ministry of Science and ICT;\n'
 '  4. The Ministry of Agriculture, Food and Rural Affairs;\n'
 '  5. The Ministry of Trade, Industry and Energy;\n'
 '  6. The Ministry of Oceans and Fisheries;\n'
 '  7. The Ministry of SMEs and Startups.')
('제5조(관계 중앙행정기관의 장의 범위) 법 제5조제3항에서 "대통령령으로 정하는 관계 중앙행정기관의 장"이란 다음 각 호의 기관의 장을 말한다.1. 기획재정부2. 교육부3. '
 '과학기술정보통신부4. 농림축산식품부5. 산업통상자원부6. 해양수산부7. 중소벤처기업부')


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

11843
('Article 20 (Grounds for Exemption)\n'
 'A court may exempt any of the following persons from duties as a juror at its discretion or at the request of the person:\n'
 '  1. A person who is not less than 70 years of age;\n'
 '  2. A person who has ever attended selection proceedings as a prospective juror during the past five years;\n'
 '  3. A person whose case prosecuted for an offense punishable by imprisonment without prison labor or any heavier punishment is still pending;\n'
 '  4. A person who is under arrest or confinement pursuant to any Act or subordinate statute;\n'
 '  5. A person whose performance of duties is likely to cause harm to him/herself or a third party or is likely to sustain irrecoverable damage to his/her career;\n'
 '  6. A person who has difficulties in making an appearance before court due to serious illness, injury, or disabilities;\n'
 '  7. A person who has difficulties in performing duties as a juror due to any other unavoidable cause or event.')
(

In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

16666
('Article 70 (Purposes of Fund) The Fund shall be used for any of the following purposes:\n'
 "  1. Creation and operation of the Government's foundation for reducing greenhouse gases;\n"
 '  2. Support for conversion of industries, labor, and local economy, and greenhouse gas reduction activities of enterprises, for the transition to a carbon neutral society and promotion of green '
 'growth;\n'
 '  3. Support for conversion and creation of jobs in the regions with worsening social and economic conditions, or for workers and social groups suffering, in the course of coping with climate '
 'crisis;\n'
 '  4. Research and development, and training of human resources, of green technology for coping with climate crisis;\n'
 '  5. Loan, investment, or other financial support necessary for coping with climate crisis;\n'
 '  6. Education and public relations for coping with climate crisis;\n'
 '  7. International cooperation for coping with climate crisis;\n'
 '  8. Repayment of the pr

In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

59123
('Article 10 (Obligation to Compensate for Damage) (1) Where any person has suffered damage from unfair labeling or advertising which violates Article 3 (1), business entities, etc. shall be '
 'obligated to compensate for such damage to the aggrieved person.\n'
 '  (2) No business entities, etc. obligated to compensate for damage pursuant to paragraph (1) shall be exempted from such obligation, on the ground that damage has not been caused by intention or '
 'negligence.\n'
 '  [This Article Wholly Amended by Act No. 11050, Sep. 15, 2011]')
('제10조(손해배상책임)① 사업자등은 제3조제1항을 위반하여 부당한 표시ㆍ광고 행위를 함으로써 피해를 입은 자가 있는 경우에는 그 피해자에 대하여 손해배상의 책임을 진다.② '
 '제1항에 따라 손해배상의 책임을 지는 사업자등은 고의 또는 과실이 없음을 들어 그 피해자에 대한 책임을 면할 수 없다.[전문개정 2011.9.15]')


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

11357
'Article 23 Deleted. <by Presidential Decree No. 20131, Jun. 28, 2007>'
'제23조 삭제 <2007.6.28>'


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

8569
('Article 10 (Display of Tourist Signs) (1) A tourism business operator may display tourist signs at his or her business facilities, as prescribed by Ordinance of the Ministry of Culture, Sports and '
 'Tourism. <Amended on Feb. 29, 2008>\n'
 '  (2) No tourism business operator shall display any false tourist sign under paragraph (1) (hereinafter referred to as “tourist sign”), or display and advertise any false information on the '
 'tourist sign. <Newly Inserted on Mar. 11, 2014>\n'
 '  (3) No person, other than a tourism business operator, shall display a tourist sign under paragraph (1) at his or her business facility, nor use any trade name that includes words indicating all '
 'or some of the tourism businesses referred to in Article 3, if such words are likely to misrepresent the person as a tourism business operator. <Amended on Mar. 11, 2014>\n'
 '  (4) The specific scope of words included in trade names of tourism businesses, which is fully or partially prohibited for us

In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

43286
('Article 9 (Bearing Expenses) The State and local governments shall bear all or part of the expenses needed for the early childhood education of the children of the welfare recipients as prescribed '
 'in the Protection of Minimum Living Standards Act or the children of the low-income families as prescribed by the Presidential Decree.')
'제9조 (비용의 부담) 국가 및 지방자치단체는 생활보호법에 의한 생활보호대상자와 대통령령이 정하는 저소득층 자녀의 유아교육에 필요한 비용의 전부 또는 일부를 부담하여야 한다.'


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

26581
('Article 14 (Duties of Executive Officers) (1) The president shall represent the Corporation, and exercise overall control over general affairs of the Corporation.\n'
 '  (2) Directors shall take charge of their respective affairs of the Corporation as prescribed by the articles of association, and, in cases of inability of the president under unavoidable '
 'circumstances, act on his/her behalf in the order of priority as determined by the articles of association.\n'
 '  (3) The auditor shall inspect and audit operations and accounting of the Corporation.\n'
 '  [This Article Wholly Amended by Act No. 8994, Mar. 28, 2008]')
('제14조(임원의 직무)① 이사장은 공단을 대표하고 공단의 업무를 총괄한다.② 이사는 정관으로 정하는 바에 따라 공단의 업무를 분장하며 이사장이 불가피한 사유로 직무를 수행할 '
 '수 없을 때에는 정관으로 정하는 순위에 따라 그 직무를 대리한다.③ 감사는 공단의 업무와 회계를 감사(監査)한다.[전문개정 2008.3.28]')


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

37940
'Article 15 Deleted. <Jul. 28, 2014>'
'제15조 삭제 <2014.7.28>'


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

34712
('Article 88 (Principle of Bearing of Expenses)\n'
 'Expenses incurred in proceedings for limiting liability and the remuneration for the administrator (hereinafter referred to as "expenses") shall be borne by the petitioner:                         '
 'Provided, That the foregoing shall not apply to cases otherwise provided for in this Act.                     \n'
 '  [This Article Wholly Amended by Act No. 9833, Dec. 29, 2009]')
('제88조(비용 부담의 원칙) 책임제한절차를 위하여 필요한 비용 및 관리인의 보수(이하 "비용등"이라 한다)는 신청인의 부담으로 한다. 다만, 이 법에 달리 규정된 경우에는 '
 '그러하지 아니하다.[전문개정 2009.12.29]')


In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

66438
('Article 273 (Abolition by Request) (1) If it becomes obvious that the company can perform in full all of the obligations to reorganization creditors and security holders who made reports within '
 'the report period, the court shall decide to abolish the reorganization proceedings upon the application of the receiver, company or reported reorganization creditors or security holders. <Amended '
 'by Act No. 6085, Dec. 31, 1999>\n'
 '  (2) The applicant shall give a clear explanation of the facts giving rise to the abolition of the reorganization proceedings referred to in the preceding paragraph.')
('제273조 (신청에 의한 폐지)①회사가 신고기간내에 신고한 모든 정리채권자와 정리담보권자에 대한 채무를 완제할 수 있음이 명백하게 된 때에는 법원은 관리인, 회사 또는 '
 '신고한 정리채권자나 정리담보권자의 신청에 의하여 정리절차폐지의 결정을 하여야 한다. <개정 1999.12.31>②신청인은 전항에 정하는 정리절차폐지의 원인인 사실을 '
 '소명하여야 한다.')


# 예외 사례

## 1

In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2])
pprint.pprint(df.iloc[num,-1])

26847
('Article 67 (Supplementation of Written Requests) (1) Where the Pension '
 'Service Agency finds any defects in a written application for the '
 'examination referred to in Article 65, it may request that an applicant or '
 'the president of the board of directors supplement them within a given '
 'period of time. \n'
 '  (2) If an applicant fails to supplement them by a given period of time '
 'prescribed in paragraph (1) without good cause and if it is not possible to '
 'conduct an examination of benefits based solely on the facts stated in the '
 'written application, the Pension Service Agency may dismiss an '
 'application. \n'
 '  [This Article Wholly Amended by Presidential Decree No. 22943, May 30, '
 '2011]')
('제67조(청구서의 보완 등)① 연금관리단은 제65조에 따른 심사청구서에서 미비사항을 발견한 경우에는 기간을 정하여 청구인 또는 이사장에게 '
 '보완을 요구할 수 있다.② 청구인이 정당한 사유 없이 제1항에 따른 보완기간 내에 보완을 하지 아니하고 청구서에 적힌 사실만으로는 '
 '급여심사를 할 수 없을 때에는 그 청구를 기각할 수 있다.[전문개정 2011.5.30]\n'
 '\t\t\n'
 '\t\t[제54조에서 이동 <2011.5.30>]')


In [ ]:
df.iloc[26847,1]

'ENFORCEMENT DECREE OF THE SPECIAL POST OFFICES ACT'

In [ ]:
kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
        + '&LM=' + str(df.iloc[26847,0])
        # + '&ID=' + str(법령ID)
kor_response = requests.get(kor_url)
kor_contents = kor_response.text
kor_dict_contents = xmltodict.parse(kor_contents)

In [ ]:
kor_dict_contents['법령']['기본정보']

{'법령ID': '003623',
 '공포일자': '20210105',
 '공포번호': '31380',
 '언어': '한글',
 '법종구분': {'@법종구분코드': 'A0007', '#text': '대통령령'},
 '법령명_한글': '별정우체국법 시행령',
 '법령명_한자': '별정우체국법 시행령',
 '법령명약칭': None,
 '제명변경여부': 'N',
 '한글법령여부': 'Y',
 '편장절관': '43010000',
 '소관부처': {'@소관부처코드': '1721000', '#text': '과학기술정보통신부'},
 '전화번호': '044-200-8187, 8188',
 '시행일자': '20210105',
 '제개정구분': '타법개정',
 '별표편집여부': 'Y',
 '공포법령여부': 'Y',
 '연락부서': {'부서단위': {'@부서키': '209015',
   '소관부처명': '과학기술정보통신부',
   '소관부처코드': '1721000',
   '부서명': '우정사업본부 기반시설기획담당관',
   '부서연락처': '044-200-8187, 8188'}}}

In [ ]:
for v in kor_dict_contents['법령']['조문']['조문단위']:
    if  v['조문번호'] == '67':
        pprint.pprint(v)

{'@조문키': '0067001',
 '조문내용': '제67조(청구서의 보완 등)',
 '조문번호': '67',
 '조문변경여부': 'N',
 '조문시행일자': '20210105',
 '조문여부': '조문',
 '조문이동이전': '000000',
 '조문이동이후': '000000',
 '조문제목': '청구서의 보완 등',
 '조문참고자료': '[전문개정 2011.5.30]\n\t\t\n\t\t[제54조에서 이동 <2011.5.30>]',
 '항': [{'항내용': '① 연금관리단은 제65조에 따른 심사청구서에서 미비사항을 발견한 경우에는 기간을 정하여 청구인 또는 이사장에게 '
               '보완을 요구할 수 있다.',
        '항번호': '①'},
       {'항내용': '② 청구인이 정당한 사유 없이 제1항에 따른 보완기간 내에 보완을 하지 아니하고 청구서에 적힌 사실만으로는 '
               '급여심사를 할 수 없을 때에는 그 청구를 기각할 수 있다.',
        '항번호': '②'}]}


In [ ]:
eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
        + '&ID=' + str('003623')
        # + '&LM=' + str('ENFORCEMENT DECREE OF THE SPECIAL POST OFFICES ACT')
        # + '&MST=' + str(df.iloc[26847,0])
eng_response = requests.get(eng_url)
eng_contents = eng_response.text
eng_dict_contents = xmltodict.parse(eng_contents)
eng_dict_contents

{'Law': '일치하는 영문법령이 없습니다.  영문법령명을 확인하여 주십시오.'}

In [ ]:
for v in eng_dict_contents['Law']['JoSection']['Jo']:
    if  v['@No'] == '67':
        pprint.pprint(v)

KeyError: 'Law'

## 2

In [ ]:
num = random.randint(0,66643)
print(num)
pprint.pprint(df.iloc[num,-2], width = 200)
pprint.pprint(df.iloc[num,-1], width = 100)

63885
('Article 32 (Direct Compulsion) (1) Direct compulsion shall be enforced where it is impracticable to secure the performance of an administrative obligation or where it is impossible to realize such '
 'obligationby means of administrative vicarious execution or the imposition of a charge for compelling compliance.\n'
 '  (2) A person in charge of enforcement who is dispatched to the scene to enforce direct compulsion shall present a certificate indicating that he or she is in charge of enforcement.\n'
 '  (3) Article 31 (3) and (4) shall apply mutatis mutandis to warnings and notices of direct compulsion.\n'
 '  [Enforcement Date: Mar. 24, 2023] Article 32')
('제32조(직접강제)① 직접강제는 행정대집행이나 이행강제금 부과의 방법으로는 행정상 의무 이행을 확보할 수 없거나 그 실현이 불가능한 경우에 실시하여야 한다.② 직접강제를 '
 '실시하기 위하여 현장에 파견되는 집행책임자는 그가 집행책임자임을 표시하는 증표를 보여 주어야 한다.③ 직접강제의 계고 및 통지에 관하여는 제31조제3항 및 제4항을 '
 '준용한다.')


In [ ]:
df.iloc[63885,0]

'행정기본법'

In [ ]:
kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
        + '&LM=' + str('행정기본법')
        # + '&ID=' + str(법령ID)
kor_response = requests.get(kor_url)
kor_contents = kor_response.text
kor_dict_contents = xmltodict.parse(kor_contents)

In [ ]:
for v in kor_dict_contents['법령']['조문']['조문단위']:
    if  v['조문번호'] == '32':
        pprint.pprint(v)

{'@조문키': '0032001',
 '조문내용': '제32조(직접강제)',
 '조문번호': '32',
 '조문변경여부': 'N',
 '조문시행일자': '20230628',
 '조문여부': '조문',
 '조문이동이전': None,
 '조문이동이후': None,
 '조문제목': '직접강제',
 '항': [{'항내용': '① 직접강제는 행정대집행이나 이행강제금 부과의 방법으로는 행정상 의무 이행을 확보할 수 없거나 그 실현이 '
               '불가능한 경우에 실시하여야 한다.',
        '항번호': '①'},
       {'항내용': '② 직접강제를 실시하기 위하여 현장에 파견되는 집행책임자는 그가 집행책임자임을 표시하는 증표를 보여 주어야 '
               '한다.',
        '항번호': '②'},
       {'항내용': '③ 직접강제의 계고 및 통지에 관하여는 제31조제3항 및 제4항을 준용한다.', '항번호': '③'}]}
